# Preparación

In [ ]:
pip install --upgrade scikit-learn

In [ ]:
pip install scikit-tda

     |████████████████████████████████| 125 kB 5.3 MB/s 
     |████████████████████████████████| 47 kB 4.0 MB/s 
     |████████████████████████████████| 72 kB 433 kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for hopcroftkarp: filename=hopcroftkarp-1.2.5-py2.py3-none-any.whl size=18119 sha256=41e3ab1432de1175d0d535653ad0d89941057d638a87a11aacdc576e5d43d429
  Stored in directory: /root/.cache/pip/wheels/d2/9f/a8/67f1b86e47cd17338d3d07939f4660378e65b758c4594f96e3
  Created wheel for ripser: filename=ripser-0.6.1-cp37-cp37m-linux_x86_64.whl size=455258 sha256=63c58c5da53232312f4cf5ef0de2afd1f96209e92673ffd077a33d3bb311bda9
  Stored in directory: /root/.cache/pip/wheels/6b/e3/c8/209a9df47cc9af8c6d2dbee4db558dc129494d3f4fbd670285
Successfully built hopcroftkarp ripser


In [ ]:
pip install umap-learn

     |████████████████████████████████| 86 kB 3.1 MB/s 
     |████████████████████████████████| 1.1 MB 37.9 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.2-py3-none-any.whl size=82709 sha256=e50d16674ae0893ed37983ca7253232857247b8de68c18c7d06942c0a2d924a6
  Stored in directory: /root/.cache/pip/wheels/84/1b/c6/aaf68a748122632967cef4dffef68224eb16798b6793257d82
  Created wheel for pynndescent: filename=pynndescent-0.5.5-py3-none-any.whl size=52603 sha256=2b4e92105f6f6ea65a1cd7e673d2ce132d036b6e4db84095d66b455f9cf731fc
  Stored in directory: /root/.cache/pip/wheels/af/e9/33/04db1436df0757c42fda8ea6796d7a8586e23c85fac355f476
Successfully built umap-learn pynndescent


In [ ]:
pip install gudhi

     |████████████████████████████████| 28.2 MB 22 kB/s 


In [ ]:
import time
import warnings
from ripser import ripser
import numpy as np
import matplotlib.pyplot as plt
from persim import plot_diagrams
import persim
import glob
from numpy import asarray
from PIL import Image
import os

from sklearn import cluster, datasets
from sklearn.preprocessing import StandardScaler
from itertools import cycle, islice
import random
from sklearn.datasets import load_breast_cancer

from sklearn.semi_supervised import LabelPropagation
from sklearn.semi_supervised import LabelSpreading
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.svm import SVC
import warnings

import umap

import pandas as pd

warnings.filterwarnings('ignore')

In [ ]:
!wget https://unirioja-my.sharepoint.com/:x:/g/personal/adines_unirioja_es/Ec1NYYxE6WRLlC0uuyXPzcgBO7w6wZcxW4WHsObhYBrHmA?download=1 -O banknote.csv

--2021-11-16 16:00:09--  https://unirioja-my.sharepoint.com/:x:/g/personal/adines_unirioja_es/Ec1NYYxE6WRLlC0uuyXPzcgBO7w6wZcxW4WHsObhYBrHmA?download=1
Resolving unirioja-my.sharepoint.com (unirioja-my.sharepoint.com)... 40.108.195.27
Connecting to unirioja-my.sharepoint.com (unirioja-my.sharepoint.com)|40.108.195.27|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /personal/adines_unirioja_es/Documents/banknote.csv [following]
--2021-11-16 16:00:10--  https://unirioja-my.sharepoint.com/personal/adines_unirioja_es/Documents/banknote.csv
Reusing existing connection to unirioja-my.sharepoint.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 46400 (45K) [application/octet-stream]
Saving to: ‘banknote.csv’

banknote.csv        100%[===================>]  45.31K  --.-KB/s    in 0.09s   

2021-11-16 16:00:10 (486 KB/s) - ‘banknote.csv’ saved [46400/46400]



In [ ]:
def load_prima():
  df=pd.read_csv("banknote.csv", sep=',',header=None)
  df1 = df.iloc[:,:-1]
  df2=df.iloc[:,-1]
  return df1.values,df2.values

In [ ]:
def preparar_dataset(data,target,n_labeled=25):
  unos=np.where(target==1)[0]
  ceros=np.where(target==0)[0]

  datos_uno=np.array(data)[unos]
  datos_cero=np.array(data)[ceros]

  unos_valid=int((len(list(unos))-n_labeled)*0.2)
  ceros_valid=int((len(list(ceros))-n_labeled)*0.2)

  random.seed(15)
  indices_unos=random.sample(list(unos),n_labeled+unos_valid)

  random.seed(10)
  indices_ceros=random.sample(list(ceros),n_labeled+ceros_valid)


  puntos_unos=np.array(data)[indices_unos[0:n_labeled]]
  puntos_ceros=np.array(data)[indices_ceros[0:n_labeled]]

  puntos_unos_valid=np.array(data)[indices_unos[n_labeled:]]
  puntos_ceros_valid=np.array(data)[indices_ceros[n_labeled:]]

  unlabeled_unos=list(set(unos)-set(indices_unos))
  unlabeled_ceros=list(set(ceros)-set(indices_ceros))
  unlabeled=unlabeled_unos+unlabeled_ceros
  X_unlabeled_unos=np.array(data)[unlabeled_unos]
  X_unlabeled_ceros=np.array(data)[unlabeled_ceros]

  return (puntos_ceros,puntos_unos,X_unlabeled_ceros,X_unlabeled_unos,puntos_ceros_valid,puntos_unos_valid)

In [ ]:
def analizar_punto(punto,puntos_ceros,puntos_unos,dgms_ceros,dgms_unos,th=0,dim=0):
  dgms_ceros_mod = ripser(np.array([punto.tolist()]+puntos_ceros.tolist()))['dgms'][dim]
  distance_bottleneck_cero = persim.bottleneck(dgms_ceros_mod, dgms_ceros, matching=False)

  dgms_unos_mod = ripser(np.array([punto.tolist()]+puntos_unos.tolist()))['dgms'][dim]
  distance_bottleneck_uno = persim.bottleneck(dgms_unos_mod, dgms_unos, matching=False)
  clase=-1

  if distance_bottleneck_cero>distance_bottleneck_uno:
    clase=1
  else:
    clase=0
  return clase

In [ ]:
def analizar_punto_th(punto,puntos_ceros,puntos_unos,dgms_ceros,dgms_unos,th=0.8,dim=0):
  dgms_ceros_mod = ripser(np.array([punto.tolist()]+puntos_ceros.tolist()))['dgms'][dim]
  distance_bottleneck_cero = persim.bottleneck(dgms_ceros_mod, dgms_ceros, matching=False)

  dgms_unos_mod = ripser(np.array([punto.tolist()]+puntos_unos.tolist()))['dgms'][dim]
  distance_bottleneck_uno = persim.bottleneck(dgms_unos_mod, dgms_unos, matching=False)

  distancia=distance_bottleneck_cero+distance_bottleneck_uno

  if max(distance_bottleneck_cero/distancia,distance_bottleneck_uno/distancia)<th:
    clase=-1
  elif distance_bottleneck_cero>distance_bottleneck_uno:
    clase=1
  else:
    clase=0
  return clase

In [ ]:
def analizar_punto_Wasserstein(punto,puntos_ceros,puntos_unos,dgms_ceros,dgms_unos,th=0,dim=0):
  dgms_ceros_mod = ripser(np.array([punto.tolist()]+puntos_ceros.tolist()))['dgms'][dim]
  distance_bottleneck_cero = persim.wasserstein(dgms_ceros_mod, dgms_ceros, matching=False)

  dgms_unos_mod = ripser(np.array([punto.tolist()]+puntos_unos.tolist()))['dgms'][dim]
  distance_bottleneck_uno = persim.wasserstein(dgms_unos_mod, dgms_unos, matching=False)
  clase=-1

  if distance_bottleneck_cero>distance_bottleneck_uno:
    clase=1
  else:
    clase=0
  return clase

In [ ]:
def analizar_punto_Wasserstein_th(punto,puntos_ceros,puntos_unos,dgms_ceros,dgms_unos,th=0.8,dim=0):
  dgms_ceros_mod = ripser(np.array([punto.tolist()]+puntos_ceros.tolist()))['dgms'][dim]
  distance_bottleneck_cero = persim.wasserstein(dgms_ceros_mod, dgms_ceros, matching=False)

  dgms_unos_mod = ripser(np.array([punto.tolist()]+puntos_unos.tolist()))['dgms'][dim]
  distance_bottleneck_uno = persim.wasserstein(dgms_unos_mod, dgms_unos, matching=False)

  distancia=distance_bottleneck_cero+distance_bottleneck_uno

  if max(distance_bottleneck_cero/distancia,distance_bottleneck_uno/distancia)<th:
    clase=-1
  elif distance_bottleneck_cero>distance_bottleneck_uno:
    clase=1
  else:
    clase=0
  return clase

In [ ]:
def analizar_puntos(data,target,func,th=0,reduccion=False,dim=0):
  (puntos_ceros,puntos_unos,X_unlabeled_ceros,X_unlabeled_unos,X_unlabeled_ceros_valid,X_unlabeled_unos_valid)=preparar_dataset(data,target)
  embedding=data
  if reduccion==True:
    embedding=umap.UMAP(random_state=75).fit_transform(data)
  (puntos_ceros_umap,puntos_unos_umap,X_unlabeled_ceros_umap,X_unlabeled_unos_umap,X_unlabeled_ceros_valid_umap,X_unlabeled_unos_valid_umap)=preparar_dataset(embedding,target)


  bien=0
  mal=0
  dudoso=0

  puntos_ceros_final=puntos_ceros.tolist()
  puntos_unos_final=puntos_unos.tolist()
  puntos_dudosos_final=[]

  dgms_ceros = ripser(puntos_ceros_umap)['dgms'][dim]
  dgms_unos = ripser(puntos_unos_umap)['dgms'][dim]
  for i,punto in enumerate(X_unlabeled_unos_umap):
    clase=func(punto,puntos_ceros_umap,puntos_unos_umap,dgms_ceros,dgms_unos,th,dim)
    if clase==1:
      bien=bien+1
      puntos_unos_final.append(X_unlabeled_unos[i])
    elif clase==0:
      mal=mal+1
      puntos_ceros_final.append(X_unlabeled_unos[i])
    else:
      dudoso=dudoso+1
      puntos_dudosos_final.append(X_unlabeled_unos[i])


  
  for i,punto in enumerate(X_unlabeled_ceros_umap):
    clase=func(punto,puntos_ceros_umap,puntos_unos_umap,dgms_ceros,dgms_unos,th,dim)
    if clase==1:
      mal=mal+1
      puntos_unos_final.append(X_unlabeled_ceros[i])
    elif clase==0:
      bien=bien+1
      puntos_ceros_final.append(X_unlabeled_ceros[i])
    else:
      dudoso=dudoso+1
      puntos_dudosos_final.append(X_unlabeled_ceros[i])
  
  comprobar_accuracy(np.array(puntos_ceros_final),np.array(puntos_unos_final),X_unlabeled_ceros_valid,X_unlabeled_unos_valid)

  return (bien, mal,dudoso,np.array(puntos_ceros_final),np.array(puntos_unos_final),np.array(puntos_dudosos_final))

In [ ]:
def resumen(bien,mal,dudosos):
  print('-----------------RESUMEN------------------')
  print('Bien anotados '+str(bien))
  print('Mal anotados '+str(mal))
  print('Sin anotar '+str(dudosos))
  print('')
  if bien+mal!=0:
    print('Porcentaje correcto '+str(bien/(bien+mal)))
  else:
    print('Porcentaje correcto 0')
  print('Porcentaje anotado '+str((bien+mal)/(bien+mal+dudosos)))

In [ ]:
import gudhi as gd  

def obtener_radio(puntos):
  skeleton = gd.RipsComplex(points = puntos, max_edge_length = 90000000000)
  simplex_tree = skeleton.create_simplex_tree()
  filt=simplex_tree.get_filtration()
  *_, last0 = filt
  return last0[1]

In [ ]:
def analizar_punto_gudhi(punto,puntos_ceros,puntos_unos,last0,last1):
  last_0=obtener_radio(np.array([punto.tolist()]+puntos_ceros.tolist()))
  last_1=obtener_radio(np.array([punto.tolist()]+puntos_unos.tolist()))


  if abs(last0-last_0)<0.00001 and abs(last1-last_1)>0.00001:
    clase=0
  elif abs(last1-last_1)<0.00001 and abs(last0-last_0)>0.00001:
    clase=1
  else:
    clase=-1
  return clase

In [ ]:
def analizar_punto_gudhi2(punto,puntos_ceros,puntos_unos,last0,last1):
  
  last_0=obtener_radio(np.array([punto.tolist()]+puntos_ceros.tolist()))
  last_1=obtener_radio(np.array([punto.tolist()]+puntos_unos.tolist()))

  dif0=abs(last0-last_0)
  dif1=abs(last1-last_1)
  if dif0==0 and dif1==0:
    clase=-1
  elif dif0<dif1:
    clase=0
  else:
    clase=1
  return clase

In [ ]:
def analizar_puntos_gudhi(data,target,func,reduccion=False):
  (puntos_ceros,puntos_unos,X_unlabeled_ceros,X_unlabeled_unos,X_unlabeled_ceros_valid,X_unlabeled_unos_valid)=preparar_dataset(data,target)
  embedding=data
  if reduccion==True:
    embedding=umap.UMAP(random_state=75).fit_transform(data)
  (puntos_ceros_umap,puntos_unos_umap,X_unlabeled_ceros_umap,X_unlabeled_unos_umap,X_unlabeled_ceros_valid_umap,X_unlabeled_unos_valid_umap)=preparar_dataset(embedding,target)


  bien=0
  mal=0
  dudoso=0

  puntos_ceros_final=puntos_ceros.tolist()
  puntos_unos_final=puntos_unos.tolist()
  puntos_dudosos_final=[]

  last0=obtener_radio(puntos_ceros_umap)
  last1=obtener_radio(puntos_unos_umap)
  for i,punto in enumerate(X_unlabeled_unos_umap):
    clase=func(punto,puntos_ceros_umap,puntos_unos_umap,last0,last1)
    if clase==1:
      bien=bien+1
      puntos_unos_final.append(X_unlabeled_unos[i])
    elif clase==0:
      mal=mal+1
      puntos_ceros_final.append(X_unlabeled_unos[i])
    else:
      dudoso=dudoso+1
      puntos_dudosos_final.append(X_unlabeled_unos[i])


  
  for i,punto in enumerate(X_unlabeled_ceros_umap):
    clase=func(punto,puntos_ceros_umap,puntos_unos_umap,last0,last1)
    if clase==1:
      mal=mal+1
      puntos_unos_final.append(X_unlabeled_ceros[i])
    elif clase==0:
      bien=bien+1
      puntos_ceros_final.append(X_unlabeled_ceros[i])
    else:
      dudoso=dudoso+1
      puntos_dudosos_final.append(X_unlabeled_ceros[i])
  
  comprobar_accuracy(np.array(puntos_ceros_final),np.array(puntos_unos_final),X_unlabeled_ceros_valid,X_unlabeled_unos_valid)

  return (bien, mal,dudoso,np.array(puntos_ceros_final),np.array(puntos_unos_final),np.array(puntos_dudosos_final))

In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

def comprobar_accuracy(puntos_ceros_final,puntos_unos_final,X_unlabeled_ceros_valid,X_unlabeled_unos_valid):
    datos=np.concatenate((puntos_ceros_final,puntos_unos_final),axis=0)
    labels=np.concatenate((np.zeros(puntos_ceros_final.shape[0]),np.ones(puntos_unos_final.shape[0])))

    modelSVMLineal = SVC(kernel="linear",probability=True,random_state=90)
    modelSVMLineal.fit(datos, labels)

    mal=np.count_nonzero(modelSVMLineal.predict(X_unlabeled_ceros_valid)!=0)
    mal=mal+np.count_nonzero(modelSVMLineal.predict(X_unlabeled_unos_valid)!=1)
    total=X_unlabeled_ceros_valid.shape[0]+X_unlabeled_unos_valid.shape[0]
    print("--------------Clasificador SVM--------------")
    resumen(total-mal,mal,0)
    print("")


    # datos=np.concatenate((puntos_ceros_final,puntos_unos_final),axis=0)
    # labels=np.concatenate((np.zeros(puntos_ceros_final.shape[0]),np.ones(puntos_unos_final.shape[0])))

    modelRandomForest = RandomForestClassifier(random_state=90)
    modelRandomForest.fit(datos, labels)

    mal=np.count_nonzero(modelRandomForest.predict(X_unlabeled_ceros_valid)!=0)
    mal=mal+np.count_nonzero(modelRandomForest.predict(X_unlabeled_unos_valid)!=1)
    total=X_unlabeled_ceros_valid.shape[0]+X_unlabeled_unos_valid.shape[0]
    print("-------------- Clasificador Random Forest--------------")
    resumen(total-mal,mal,0)
    print("")

In [ ]:
def label_prop(data,target):
  (puntos_ceros,puntos_unos,X_unlabeled_ceros,X_unlabeled_unos,puntos_ceros_valid,puntos_unos_valid)=preparar_dataset(data,target)
  datos=np.concatenate((puntos_ceros,puntos_unos),axis=0)
  labels=np.concatenate((np.zeros(puntos_ceros.shape[0]),np.ones(puntos_unos.shape[0])))

  label_prop_model = LabelPropagation(kernel='knn')
  label_prop_model.fit(datos, labels)

  unos=np.where(label_prop_model.predict(X_unlabeled_ceros)==1)[0]
  ceros=np.where(label_prop_model.predict(X_unlabeled_ceros)==0)[0]


  datos_uno=np.array(X_unlabeled_ceros)[unos]
  datos_cero=np.array(X_unlabeled_ceros)[ceros]

  nuevos_puntos_cero=np.asarray(list(puntos_ceros)+list(datos_cero))
  nuevos_puntos_uno=np.asarray(list(puntos_unos)+list(datos_uno))

  unos=np.where(label_prop_model.predict(X_unlabeled_unos)==1)[0]
  ceros=np.where(label_prop_model.predict(X_unlabeled_unos)==0)[0]
  datos_uno=np.array(X_unlabeled_unos)[unos]
  datos_cero=np.array(X_unlabeled_unos)[ceros]

  nuevos_puntos_cero=np.asarray(list(nuevos_puntos_cero)+list(datos_cero))
  nuevos_puntos_uno=np.asarray(list(nuevos_puntos_uno)+list(datos_uno))

  mal=np.count_nonzero(label_prop_model.predict(X_unlabeled_ceros)!=0)
  mal=mal+np.count_nonzero(label_prop_model.predict(X_unlabeled_unos)!=1)
  total=X_unlabeled_ceros.shape[0]+X_unlabeled_unos.shape[0]
  # resumen(total-mal,mal)

  comprobar_accuracy(nuevos_puntos_cero,nuevos_puntos_uno,puntos_ceros_valid,puntos_unos_valid)

  return (total-mal, mal, 0, nuevos_puntos_cero, nuevos_puntos_uno, []) 

In [ ]:
def label_spread(data,target):
  (puntos_ceros,puntos_unos,X_unlabeled_ceros,X_unlabeled_unos,puntos_ceros_valid,puntos_unos_valid)=preparar_dataset(data,target)
  datos=np.concatenate((puntos_ceros,puntos_unos),axis=0)
  labels=np.concatenate((np.zeros(puntos_ceros.shape[0]),np.ones(puntos_unos.shape[0])))

  label_prop_model = LabelSpreading(kernel='knn')
  label_prop_model.fit(datos, labels)

  unos=np.where(label_prop_model.predict(X_unlabeled_ceros)==1)[0]
  ceros=np.where(label_prop_model.predict(X_unlabeled_ceros)==0)[0]


  datos_uno=np.array(X_unlabeled_ceros)[unos]
  datos_cero=np.array(X_unlabeled_ceros)[ceros]

  nuevos_puntos_cero=np.asarray(list(puntos_ceros)+list(datos_cero))
  nuevos_puntos_uno=np.asarray(list(puntos_unos)+list(datos_uno))

  unos=np.where(label_prop_model.predict(X_unlabeled_unos)==1)[0]
  ceros=np.where(label_prop_model.predict(X_unlabeled_unos)==0)[0]
  datos_uno=np.array(X_unlabeled_unos)[unos]
  datos_cero=np.array(X_unlabeled_unos)[ceros]

  nuevos_puntos_cero=np.asarray(list(nuevos_puntos_cero)+list(datos_cero))
  nuevos_puntos_uno=np.asarray(list(nuevos_puntos_uno)+list(datos_uno))

  mal=np.count_nonzero(label_prop_model.predict(X_unlabeled_ceros)!=0)
  mal=mal+np.count_nonzero(label_prop_model.predict(X_unlabeled_unos)!=1)
  total=X_unlabeled_ceros.shape[0]+X_unlabeled_unos.shape[0]
  # resumen(total-mal,mal)

  comprobar_accuracy(nuevos_puntos_cero,nuevos_puntos_uno,puntos_ceros_valid,puntos_unos_valid)

  return (total-mal, mal, 0, nuevos_puntos_cero, nuevos_puntos_uno, []) 

In [ ]:
def self_train(data,target):
  (puntos_ceros,puntos_unos,X_unlabeled_ceros,X_unlabeled_unos,puntos_ceros_valid,puntos_unos_valid)=preparar_dataset(data,target)
  datos=np.concatenate((puntos_ceros,puntos_unos),axis=0)
  labels=np.concatenate((np.zeros(puntos_ceros.shape[0]),np.ones(puntos_unos.shape[0])))

  svc = SVC(probability=True, gamma="auto")
  self_training_model = SelfTrainingClassifier(svc)
  self_training_model.fit(datos, labels)

  unos=np.where(self_training_model.predict(X_unlabeled_ceros)==1)[0]
  ceros=np.where(self_training_model.predict(X_unlabeled_ceros)==0)[0]


  datos_uno=np.array(X_unlabeled_ceros)[unos]
  datos_cero=np.array(X_unlabeled_ceros)[ceros]

  nuevos_puntos_cero=np.asarray(list(puntos_ceros)+list(datos_cero))
  nuevos_puntos_uno=np.asarray(list(puntos_unos)+list(datos_uno))

  unos=np.where(self_training_model.predict(X_unlabeled_unos)==1)[0]
  ceros=np.where(self_training_model.predict(X_unlabeled_unos)==0)[0]
  datos_uno=np.array(X_unlabeled_unos)[unos]
  datos_cero=np.array(X_unlabeled_unos)[ceros]

  nuevos_puntos_cero=np.asarray(list(nuevos_puntos_cero)+list(datos_cero))
  nuevos_puntos_uno=np.asarray(list(nuevos_puntos_uno)+list(datos_uno))

  mal=np.count_nonzero(self_training_model.predict(X_unlabeled_ceros)!=0)
  mal=mal+np.count_nonzero(self_training_model.predict(X_unlabeled_unos)!=1)
  total=X_unlabeled_ceros.shape[0]+X_unlabeled_unos.shape[0]
  # resumen(total-mal,mal)

  comprobar_accuracy(nuevos_puntos_cero,nuevos_puntos_uno,puntos_ceros_valid,puntos_unos_valid)

  return (total-mal, mal, 0, nuevos_puntos_cero, nuevos_puntos_uno, [])  

# Bottleneck

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 257
Mal anotados 7
Sin anotar 0

Porcentaje correcto 0.9734848484848485
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 239
Mal anotados 25
Sin anotar 0

Porcentaje correcto 0.9053030303030303
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 955
Mal anotados 103
Sin anotar 0

Porcentaje correcto 0.9026465028355387
Porcentaje anotado 1.0


# Bottleneck threshold

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_th,0.8)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 262
Mal anotados 2
Sin anotar 0

Porcentaje correcto 0.9924242424242424
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 244
Mal anotados 20
Sin anotar 0

Porcentaje correcto 0.9242424242424242
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 160
Mal anotados 0
Sin anotar 898

Porcentaje correcto 1.0
Porcentaje anotado 0.15122873345935728


# Bottleneck threshold 0.6

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_th,0.6)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 262
Mal anotados 2
Sin anotar 0

Porcentaje correcto 0.9924242424242424
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 241
Mal anotados 23
Sin anotar 0

Porcentaje correcto 0.9128787878787878
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 830
Mal anotados 65
Sin anotar 163

Porcentaje correcto 0.9273743016759777
Porcentaje anotado 0.8459357277882797


# Bottleneck threshold 0.4

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_th,0.4)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 257
Mal anotados 7
Sin anotar 0

Porcentaje correcto 0.9734848484848485
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 239
Mal anotados 25
Sin anotar 0

Porcentaje correcto 0.9053030303030303
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 955
Mal anotados 103
Sin anotar 0

Porcentaje correcto 0.9026465028355387
Porcentaje anotado 1.0


# Bottleneck threshold 0.2

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_th,0.2)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 257
Mal anotados 7
Sin anotar 0

Porcentaje correcto 0.9734848484848485
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 239
Mal anotados 25
Sin anotar 0

Porcentaje correcto 0.9053030303030303
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 955
Mal anotados 103
Sin anotar 0

Porcentaje correcto 0.9026465028355387
Porcentaje anotado 1.0


# Bottleneck UMAP

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto,0,True)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 257
Mal anotados 7
Sin anotar 0

Porcentaje correcto 0.9734848484848485
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 254
Mal anotados 10
Sin anotar 0

Porcentaje correcto 0.9621212121212122
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 1038
Mal anotados 20
Sin anotar 0

Porcentaje correcto 0.9810964083175804
Porcentaje anotado 1.0


# Bottleneck UMAP threshold

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_th,0.8,True)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 257
Mal anotados 7
Sin anotar 0

Porcentaje correcto 0.9734848484848485
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 249
Mal anotados 15
Sin anotar 0

Porcentaje correcto 0.9431818181818182
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 879
Mal anotados 0
Sin anotar 179

Porcentaje correcto 1.0
Porcentaje anotado 0.8308128544423441


# Bottleneck UMAP threshold 0.6

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_th,0.6,True)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 257
Mal anotados 7
Sin anotar 0

Porcentaje correcto 0.9734848484848485
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 250
Mal anotados 14
Sin anotar 0

Porcentaje correcto 0.946969696969697
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 960
Mal anotados 27
Sin anotar 71

Porcentaje correcto 0.9726443768996961
Porcentaje anotado 0.9328922495274102


# Bottleneck UMAP threshold 0.4

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_th,0.4,True)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 257
Mal anotados 7
Sin anotar 0

Porcentaje correcto 0.9734848484848485
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 255
Mal anotados 9
Sin anotar 0

Porcentaje correcto 0.9659090909090909
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 1036
Mal anotados 22
Sin anotar 0

Porcentaje correcto 0.9792060491493384
Porcentaje anotado 1.0


# Bottleneck UMAP threshold 0.2

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_th,0.2,True)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 257
Mal anotados 7
Sin anotar 0

Porcentaje correcto 0.9734848484848485
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 259
Mal anotados 5
Sin anotar 0

Porcentaje correcto 0.9810606060606061
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 1053
Mal anotados 5
Sin anotar 0

Porcentaje correcto 0.995274102079395
Porcentaje anotado 1.0


# Wasserstein

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_Wasserstein)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 256
Mal anotados 8
Sin anotar 0

Porcentaje correcto 0.9696969696969697
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 254
Mal anotados 10
Sin anotar 0

Porcentaje correcto 0.9621212121212122
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 1005
Mal anotados 53
Sin anotar 0

Porcentaje correcto 0.9499054820415879
Porcentaje anotado 1.0


# Wasserstein threshold

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_Wasserstein_th,0.8)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 257
Mal anotados 7
Sin anotar 0

Porcentaje correcto 0.9734848484848485
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 237
Mal anotados 27
Sin anotar 0

Porcentaje correcto 0.8977272727272727
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 185
Mal anotados 0
Sin anotar 873

Porcentaje correcto 1.0
Porcentaje anotado 0.17485822306238186


# Wasserstein threshold 0.6

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_Wasserstein_th,0.6)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 262
Mal anotados 2
Sin anotar 0

Porcentaje correcto 0.9924242424242424
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 247
Mal anotados 17
Sin anotar 0

Porcentaje correcto 0.9356060606060606
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 771
Mal anotados 7
Sin anotar 280

Porcentaje correcto 0.9910025706940874
Porcentaje anotado 0.7353497164461248


# Wasserstein threshold 0.4

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_Wasserstein_th,0.4)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 256
Mal anotados 8
Sin anotar 0

Porcentaje correcto 0.9696969696969697
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 254
Mal anotados 10
Sin anotar 0

Porcentaje correcto 0.9621212121212122
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 1005
Mal anotados 53
Sin anotar 0

Porcentaje correcto 0.9499054820415879
Porcentaje anotado 1.0


# Wasserstein threshold 0.2

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_Wasserstein_th,0.2)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 256
Mal anotados 8
Sin anotar 0

Porcentaje correcto 0.9696969696969697
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 254
Mal anotados 10
Sin anotar 0

Porcentaje correcto 0.9621212121212122
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 1005
Mal anotados 53
Sin anotar 0

Porcentaje correcto 0.9499054820415879
Porcentaje anotado 1.0


# Wasserstein UMAP

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_Wasserstein,0,True)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 256
Mal anotados 8
Sin anotar 0

Porcentaje correcto 0.9696969696969697
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 253
Mal anotados 11
Sin anotar 0

Porcentaje correcto 0.9583333333333334
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 1037
Mal anotados 21
Sin anotar 0

Porcentaje correcto 0.9801512287334594
Porcentaje anotado 1.0


# Wasserstein UMAP threshold

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_Wasserstein_th,0.8,True)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 257
Mal anotados 7
Sin anotar 0

Porcentaje correcto 0.9734848484848485
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 251
Mal anotados 13
Sin anotar 0

Porcentaje correcto 0.9507575757575758
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 827
Mal anotados 7
Sin anotar 224

Porcentaje correcto 0.9916067146282974
Porcentaje anotado 0.7882797731568998


# Wasserstein UMAP threshold 0.6

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_Wasserstein_th,0.6,True)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 256
Mal anotados 8
Sin anotar 0

Porcentaje correcto 0.9696969696969697
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 252
Mal anotados 12
Sin anotar 0

Porcentaje correcto 0.9545454545454546
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 1027
Mal anotados 7
Sin anotar 24

Porcentaje correcto 0.9932301740812379
Porcentaje anotado 0.9773156899810964


# Wasserstein UMAP threshold 0.4

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_Wasserstein_th,0.2,True)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 255
Mal anotados 9
Sin anotar 0

Porcentaje correcto 0.9659090909090909
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 243
Mal anotados 21
Sin anotar 0

Porcentaje correcto 0.9204545454545454
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 961
Mal anotados 97
Sin anotar 0

Porcentaje correcto 0.9083175803402647
Porcentaje anotado 1.0


# Wasserstein UMAP threshold 0.2

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos(data,target,analizar_punto_Wasserstein_th,0.2,True)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 255
Mal anotados 9
Sin anotar 0

Porcentaje correcto 0.9659090909090909
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 244
Mal anotados 20
Sin anotar 0

Porcentaje correcto 0.9242424242424242
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 974
Mal anotados 84
Sin anotar 0

Porcentaje correcto 0.9206049149338374
Porcentaje anotado 1.0


# Gudhi

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos_gudhi(data,target,analizar_punto_gudhi)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 247
Mal anotados 17
Sin anotar 0

Porcentaje correcto 0.9356060606060606
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 232
Mal anotados 32
Sin anotar 0

Porcentaje correcto 0.8787878787878788
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 223
Mal anotados 14
Sin anotar 821

Porcentaje correcto 0.9409282700421941
Porcentaje anotado 0.22400756143667297


# Gudhi UMAP

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos_gudhi(data,target,analizar_punto_gudhi,True)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 212
Mal anotados 52
Sin anotar 0

Porcentaje correcto 0.803030303030303
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 225
Mal anotados 39
Sin anotar 0

Porcentaje correcto 0.8522727272727273
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 116
Mal anotados 57
Sin anotar 885

Porcentaje correcto 0.6705202312138728
Porcentaje anotado 0.16351606805293006


# Gudhi 2

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos_gudhi(data,target,analizar_punto_gudhi2)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 247
Mal anotados 17
Sin anotar 0

Porcentaje correcto 0.9356060606060606
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 231
Mal anotados 33
Sin anotar 0

Porcentaje correcto 0.875
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 250
Mal anotados 14
Sin anotar 794

Porcentaje correcto 0.946969696969697
Porcentaje anotado 0.2495274102079395


# Gudhi 2 UMAP

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=analizar_puntos_gudhi(data,target,analizar_punto_gudhi2,True)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 142
Mal anotados 122
Sin anotar 0

Porcentaje correcto 0.5378787878787878
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 198
Mal anotados 66
Sin anotar 0

Porcentaje correcto 0.75
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 41
Mal anotados 77
Sin anotar 940

Porcentaje correcto 0.3474576271186441
Porcentaje anotado 0.11153119092627599


# LabelPropagation

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=label_prop(data,target)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 257
Mal anotados 7
Sin anotar 0

Porcentaje correcto 0.9734848484848485
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 246
Mal anotados 18
Sin anotar 0

Porcentaje correcto 0.9318181818181818
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 991
Mal anotados 67
Sin anotar 0

Porcentaje correcto 0.9366729678638941
Porcentaje anotado 1.0


# LabelSpreading

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=label_spread(data,target)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 257
Mal anotados 7
Sin anotar 0

Porcentaje correcto 0.9734848484848485
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 246
Mal anotados 18
Sin anotar 0

Porcentaje correcto 0.9318181818181818
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 991
Mal anotados 67
Sin anotar 0

Porcentaje correcto 0.9366729678638941
Porcentaje anotado 1.0


# SelfTraining

In [ ]:
(data,target) = load_prima()
(bien,mal,dudoso,pcero,puno,pdudoso)=self_train(data,target)
resumen(bien,mal,dudoso)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 251
Mal anotados 13
Sin anotar 0

Porcentaje correcto 0.9507575757575758
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 247
Mal anotados 17
Sin anotar 0

Porcentaje correcto 0.9356060606060606
Porcentaje anotado 1.0

-----------------RESUMEN------------------
Bien anotados 1012
Mal anotados 46
Sin anotar 0

Porcentaje correcto 0.9565217391304348
Porcentaje anotado 1.0


# Base

In [37]:
(data,target) = load_prima()
(puntos_ceros,puntos_unos,X_unlabeled_ceros,X_unlabeled_unos,puntos_ceros_valid,puntos_unos_valid)=preparar_dataset(data,target)
comprobar_accuracy(puntos_ceros,puntos_unos,puntos_ceros_valid,puntos_unos_valid)

--------------Clasificador SVM--------------
-----------------RESUMEN------------------
Bien anotados 256
Mal anotados 8
Sin anotar 0

Porcentaje correcto 0.9696969696969697
Porcentaje anotado 1.0

-------------- Clasificador Random Forest--------------
-----------------RESUMEN------------------
Bien anotados 234
Mal anotados 30
Sin anotar 0

Porcentaje correcto 0.8863636363636364
Porcentaje anotado 1.0

